In [14]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math

In [15]:
# Local on Windows 10 box
WD = os.path.join("E:\\", "BUSTEDS-MH")

tag = "empirical_unmasked_selectome"

# Additional declares
BUSTEDS_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS")
BUSTEDS_MH_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS-MH")

OUTPUT_CSV = os.path.join(WD, "tables", "Table_" + tag.upper() + "_BUSTEDS_and_BUSTEDS-MH.csv")

ER_Threshold = 5

In [16]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        print("# -- Error -- file is empty:", filename)
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

In [17]:
BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))


# Number of BUSTEDS results: 13307
# Number of BUSTEDS-MH results: 13299


## Look over BUSTEDS-MH Files

In [18]:
df_dict = {}

for item in tqdm(BUSTEDS_MH_DIR_FILES):
    basename = os.path.basename(item).replace(".nex.BUSTEDS-MH.json", "")
    # Read 
    json_data_BUSTEDS_MH = read_json(item)
    
    if json_data_BUSTEDS_MH == []: continue
    
    #print("# Data loaded")
    df_dict[basename] = {"Method": "BUSTEDS-MH"}
    df_dict[basename].update({"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]})
    df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS_MH["test results"]["p-value"]})

    # cAIC
    df_dict[basename].update({"cAIC": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]})
    
    # CV of omega
    A = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    #df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    # CV of alpha
    D = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    #df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    
    #Omegas and proportions
    data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
    w1 = round(data["0"]["omega"], 4)
    p1 = round(data["0"]["proportion"], 4)
    w2 = round(data["1"]["omega"], 4)
    p2 = round(data["1"]["proportion"], 4)
    w3 = round(data["2"]["omega"], 4)
    p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"w1": w1, "p1": p1})
    df_dict[basename].update({"w2": w2, "p2": p2})
    df_dict[basename].update({"w3": w3, "p3": p3})
    
    # SRV rates and proportions
    data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    s1 = round(data["0"]["rate"], 4)
    s_p1 = round(data["0"]["proportion"], 4)
    s2 = round(data["1"]["rate"], 4)
    s_p2 = round(data["1"]["proportion"], 4)
    s3 = round(data["2"]["rate"], 4)
    s_p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
    df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
    df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})
    
    # DH rate, TH rate, TH_SI rate
    df_dict[basename].update({"DH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 2 nucleotides are changed instantly within a single codon"])})
    df_dict[basename].update({"TH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon"])})
    df_dict[basename].update({"TH_Rate_SI": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})

    # ER Sites, thresholded
    ER_SITES = []
    ER_df_dict = {}
    if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
        #print("# ER Constrained Sites:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
            #end if
        #end for
        # add assert that there are more than 0 sites here.
        df_dict[basename].update({"BUSTEDS-MH_num_ER_Sites":  len(ER_df_dict.keys())})
        x = ER_df_dict.keys()
        x = [str(x) for x in x]
        df_dict[basename].update({"BUSTEDS-MH_ER_Sites":  "|".join(x)})
        #print(ER_df_dict.keys())
    #end if 
# end for

df_MH = pd.DataFrame.from_dict(df_dict, orient="index")
df_MH = df_MH.reset_index()
df_MH.index += 1
df_MH.rename(columns={'index': 'Gene'}, inplace = True)
#df_MH

100%|███████████████████████████████████████████████████████████████████████████| 13299/13299 [00:22<00:00, 581.48it/s]


## Look over BUSTEDS Files

In [19]:
df_dict = {}

for item in tqdm(BUSTEDS_DIR_FILES):
    basename = os.path.basename(item).replace(".nex.BUSTEDS.json", "")
    # Read json
    #print()
    json_data_BUSTEDS = read_json(item)
    #print("# Data loaded:", item)
    
    df_dict[basename] = {"Method": "BUSTEDS"}
    df_dict[basename].update({"Sequences": json_data_BUSTEDS["input"]["number of sequences"]})
    df_dict[basename].update({"Codons": json_data_BUSTEDS["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS["test results"]["p-value"]})

    # cAIC
    df_dict[basename].update({"cAIC": json_data_BUSTEDS["fits"]["Unconstrained model"]["AIC-c"]})
    
    A = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    #df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    #df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    #Omegas and proportions
    data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
    w1 = round(data["0"]["omega"], 4)
    p1 = round(data["0"]["proportion"], 4)
    w2 = round(data["1"]["omega"], 4)
    p2 = round(data["1"]["proportion"], 4)
    w3 = round(data["2"]["omega"], 4)
    p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"w1": w1, "p1": p1})
    df_dict[basename].update({"w2": w2, "p2": p2})
    df_dict[basename].update({"w3": w3, "p3": p3})
    
    # SRV rates and proportions
    data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    s1 = round(data["0"]["rate"], 4)
    s_p1 = round(data["0"]["proportion"], 4)
    s2 = round(data["1"]["rate"], 4)
    s_p2 = round(data["1"]["proportion"], 4)
    s3 = round(data["2"]["rate"], 4)
    s_p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
    df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
    df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})
    
    # ER Sites
    ER_SITES = []
    ER_df_dict = {}
    
    if "constrained" in json_data_BUSTEDS["Evidence Ratios"].keys():
        #print("# ER Constrained Sites:", len(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS ER": val}
            #end if
        #end for
        df_dict[basename].update({"num_ER_Sites":  int(len(ER_df_dict.keys()))})
        df_dict[basename].update({"BUSTEDS_num_ER_Sites":  len(ER_df_dict.keys())})
        x = ER_df_dict.keys()
        x = [str(x) for x in x]
        df_dict[basename].update({"BUSTEDS_ER_Sites":  "|".join(x)})
        #print(ER_df_dict.keys())
    #end if   
    

# End for

df = pd.DataFrame.from_dict(df_dict, orient="index")
df = df.reset_index()
df.index += 1
df.rename(columns={'index': 'Gene'}, inplace = True)
#df

100%|███████████████████████████████████████████████████████████████████████████| 13307/13307 [00:39<00:00, 337.05it/s]


## Calculate cAIC statistics

In [25]:
df["ΔcAIC"] = ""
df["RelativeSupport"] = ""
df["ER_Sites_Intersection"] = ""

for index, row in tqdm(df_MH.iterrows()):
    #MH_cAIC = df_MH[]
    #df_temp = df[df["Gene"] == gene]
    #print(df_temp)
    #print(row["Gene"], row["cAIC"])    
    gene = row["Gene"]
    MH_cAIC = float(row["cAIC"])
    
    BUSTEDS_row = df[df["Gene"] == row["Gene"]]
    # df[df['LastName'] == 'Smith'].index
    index_BUSTEDS = df[df["Gene"] == row["Gene"]].index
    try:
        BUSTEDS_cAIC = float(BUSTEDS_row["cAIC"])
    except:
        print(BUSTEDS_row)
        pass
    #end try
    
    #print(float(BUSTEDS_cAIC["cAIC"]))
    best_model = min(MH_cAIC, BUSTEDS_cAIC)
    #print()
    #print("# Gene:", row["Gene"])

    if BUSTEDS_cAIC == best_model:
        which_is_best = "BUSTEDS"
        delta_cAIC = MH_cAIC - best_model
        relative_support = math.exp(-delta_cAIC/2)
        # add to table
        #df.at['C', 'x'] = 10
        # 
        df.at[index_BUSTEDS_cAIC, "ΔcAIC"] = delta_cAIC
        df.at[index_BUSTEDS_cAIC, "RelativeSupport"] = relative_support
    elif MH_cAIC == best_model:
        which_is_best = "BUSTEDS-MH"
        delta_cAIC = BUSTEDS_cAIC - best_model
        relative_support = math.exp(-delta_cAIC/2)
        df_MH.at[index, "ΔcAIC"] = delta_cAIC
        df_MH.at[index, "RelativeSupport"] = relative_support
    else:
        pass
    #end if
    #print("# Best model is:", best_model, which_is_best, "by", delta_cAIC)
    #print("# With relative support:", relative_support)
    
    # Intersections of ER Sites.
    #print("# Examining ER Sites")
    # BUSTEDS-MH_ER_Sites
    # BUSTEDS_ER_Sites
    try:
        BUSTEDS_MH_ER_Sites = row["BUSTEDS-MH_ER_Sites"].split("|")
        BUSTEDS_df = df[df["Gene"] == row["Gene"]]
        BUSTEDS_ER_Sites    = BUSTEDS_df["BUSTEDS_ER_Sites"].tolist()[0].split("|")
        #print(BUSTEDS_MH_ER_Sites, BUSTEDS_ER_Sites)
        intersection = set(BUSTEDS_MH_ER_Sites).intersection(BUSTEDS_ER_Sites)
        #print(intersection)
        df.at[index, "ER_Sites_Intersection"] = "|".join(intersection)
    except:
        #print("ERROR --", row["BUSTEDS-MH_ER_Sites"])
        pass
    #end try
#end for
    

10807it [00:35, 338.49it/s]

Empty DataFrame
Columns: [Gene, Method, Sequences, Codons, LRT p-value, cAIC, w1, p1, w2, p2, w3, p3, SRV1, SRV_p1, SRV2, SRV_p2, SRV3, SRV_p3, num_ER_Sites, BUSTEDS_num_ER_Sites, BUSTEDS_ER_Sites, ΔcAIC, RelativeSupport, ER_Sites_Intersection]
Index: []

[0 rows x 24 columns]


13299it [00:43, 306.97it/s]


## Concat tables


In [26]:
#df = df.sort_values(by="Sequences", ascending=False)
#df_MH = df_MH.sort_values(by="Sequences", ascending=False)

result = pd.concat([df_MH, df])
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1
result

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,w1,p1,w2,p2,...,TH_Rate,TH_Rate_SI,BUSTEDS-MH_num_ER_Sites,BUSTEDS-MH_ER_Sites,ΔcAIC,RelativeSupport,num_ER_Sites,BUSTEDS_num_ER_Sites,BUSTEDS_ER_Sites,ER_Sites_Intersection
1,ENSGT00390000000002.Euteleostomi.001,BUSTEDS,24,428,0.500000,15230.383413,0.0083,0.8481,0.2237,0.1472,...,,,,,,,0.0,0.0,,
2,ENSGT00390000000002.Euteleostomi.001,BUSTEDS-MH,24,428,0.453178,15218.395700,0.0108,0.8748,0.2630,0.1247,...,0.0,10.81894,0.0,,11.987712,0.002494,,,,
3,ENSGT00390000000002.Euteleostomi.002,BUSTEDS,35,520,0.500000,28163.498555,0.0171,0.9297,0.2579,0.0703,...,,,,,,,,,,
4,ENSGT00390000000002.Euteleostomi.002,BUSTEDS-MH,35,520,0.500000,28167.958840,0.0288,1.0000,0.0487,0.0000,...,0.089589,0.0,,,,,,,,
5,ENSGT00390000000002.Euteleostomi.003,BUSTEDS,30,381,0.342345,18555.324235,0.0113,0.4917,0.0967,0.4930,...,,,,,,,0.0,0.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26602,ENSGT00680000101373.Euteleostomi.001,BUSTEDS-MH,6,438,0.500000,3894.546024,0.0363,0.0000,0.3503,1.0000,...,0.0,0.0,,,,,,,,
26603,ENSGT00680000101376.Euteleostomi.001,BUSTEDS,6,344,0.500000,2828.089034,0.0000,0.8940,0.0000,0.1060,...,,,,,,,,,,
26604,ENSGT00680000101376.Euteleostomi.001,BUSTEDS-MH,6,344,0.500000,2834.279550,0.0000,1.0000,0.0006,0.0000,...,0.190025,0.335737,,,,,,,,
26605,ENSGT00680000101377.Euteleostomi.001,BUSTEDS,6,405,0.015805,4896.192723,0.1444,0.9866,0.1454,0.0032,...,,,,,,,1.0,1.0,371,


In [ ]:
"""dfv = result
dfv = dfv[['Gene', 'Sequences', 'Method', 'Codons', 'LRT p-value', 'cAIC', 'delta cAIC (best model)', 'Relative support',
       'CV(omega)', 'CV(alpha)', 'omega_3', 'proportion_3', 'DH_Rate',
       'TH_Rate', 'TH_Rate_SI', 'num_ER_Sites']]

dfv = dfv.fillna("")
dfv = dfv.sort_values(by=["Gene", "Method"], ascending=True)
dfv = dfv.reset_index(drop=True)
dfv.index += 1
"""

styled_table = result.style.background_gradient()
styled_table

## Save table

In [ ]:
print("Saving results to:", OUTPUT_CSV)
result.to_csv(OUTPUT_CSV, index=False)

## End of file

In [12]:
# Note Negative delta LL are convergence problems

In [13]:
# Lower AIC values indicate a better-fit model, and a model with a delta-AIC (the difference between the two AIC values being compared) of more than -2 is considered significantly better than the model it is being compared to

In [14]:
# Earth Mover's (Kantorovich) distance between two distrbuitions if you want a single number

In [7]:
df_dict = {}

# Set up header
with open('13Datasets_ER_Sites.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['Datafile', 'Count BUSTEDS-MH Sites',
                         'BUSTEDS-MH Sites', 'Count', 
                         'BUSTEDS Sites', "Count Intersection Sites", "Intersection Sites",
                        "Count of BUSTEDS-MH Unique sites", "BUSTEDS-MH Unique sites",
                        "Count of BUSTEDS Unique sites", "BUSTEDS Unique sites"])
#end with

for item in BUSTEDS_MH_DIR_FILES:
    basename = os.path.basename(item).replace(".nex.BUSTEDS-MH.json", "")
    
    # Find BUSTEDS File
    BUSTEDS_File = os.path.join(BUSTEDS_DIR, basename + ".nex.BUSTEDS.json")
    
    # Read json
    json_data_BUSTEDS_MH = read_json(item)
    json_data_BUSTEDS = read_json(BUSTEDS_File)
    
    print("# Data loaded")
    df_dict[basename] = {"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]}
    df_dict[basename].update({"Method": "BUSTEDS-MH"})
    df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS_MH["test results"]["p-value"]})

    A = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    D = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    df_dict[basename].update({"omega_3": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["omega"]})
    df_dict[basename].update({"proportion_3": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["proportion"]})
    
    # DH rate, TH rate
    df_dict[basename].update({"DH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 2 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate_SI": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"]})
    
    # add row.   

    # This effectively becomes a new row
    # BUSTEDS
    df_dict[basename].update({"PValue_BUSTEDS": json_data_BUSTEDS["test results"]["p-value"]})
    
    A = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    
    df_dict[basename].update({"CV(omega)_BUSTEDS": cv([A, B, C])})
    
    D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    
    df_dict[basename].update({"CV(alpha)_BUSTEDS": cv([D, E, F])})
    df_dict[basename].update({"omega_3_BUSTEDS": json_data_BUSTEDS["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["omega"]})
    df_dict[basename].update({"proportion_3_BUSTEDS": json_data_BUSTEDS["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["proportion"]})
    
    # cAIC
    AICc_BUSTEDS_MH = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]
    AICc_BUSTEDS = json_data_BUSTEDS["fits"]["Unconstrained model"]["AIC-c"]
    print("# Calculating 'Unconstrained model' AICc delta, (BUSTEDS-MH)", AICc_BUSTEDS_MH, "and (BUSTEDS)", AICc_BUSTEDS)
    delta_AICc = AICc_BUSTEDS_MH - AICc_BUSTEDS
    df_dict[basename].update({"delta_cAIC": delta_AICc})
    
    # ER Sites > 5
    ER_SITES_BUSTEDS = []
    ER_SITES_BUSTEDS_MH = []
    ER_Threshold = 1
    
    ER_df_dict = {}
    
    if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
        print("# ER Constrained:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES_BUSTEDS_MH.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
            #end if
        #end for
    #end if
    
    if "constrained" in json_data_BUSTEDS["Evidence Ratios"].keys():
        print("# ER Constrained:", len(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES_BUSTEDS.append(str(site + 1))
                if site + 1 in ER_df_dict.keys():
                    ER_df_dict[site + 1].update({"BUSTEDS ER": val})
                else:
                    ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
            #end if
        #end for
    #end if
    
    
    # This needs to be in a separate file
    print("# All BUSTEDS-MH ER Sites:",ER_SITES_BUSTEDS_MH, len(ER_SITES_BUSTEDS_MH))
    print("# All BUSTEDS ER Sites:", ER_SITES_BUSTEDS, len(ER_SITES_BUSTEDS))
    
    #union_of_sites = matches = [item for item in ER_SITES_BUSTEDS_MH if item in ER_SITES_BUSTEDS]
    #print(union_of_sites, len(union_of_sites))
    
    intersection = set(ER_SITES_BUSTEDS_MH).intersection(ER_SITES_BUSTEDS)
    print("# Intersection (where they both agree):", intersection, len(intersection))
    
    unique_to_BUSTEDS_MH = [str(x) for x in ER_SITES_BUSTEDS_MH if x not in ER_SITES_BUSTEDS]
    unique_to_BUSTEDS = [str(x) for x in ER_SITES_BUSTEDS if x not in ER_SITES_BUSTEDS_MH]
    
    print(unique_to_BUSTEDS_MH, len(unique_to_BUSTEDS_MH))
    print(unique_to_BUSTEDS, len(unique_to_BUSTEDS))
    
    with open('13Datasets_ER_Sites.csv', 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        spamwriter.writerow([basename, str(len(ER_SITES_BUSTEDS_MH)), "|".join(ER_SITES_BUSTEDS_MH), 
                                       str(len(ER_SITES_BUSTEDS)), "|".join(ER_SITES_BUSTEDS),
                                       str(len(intersection)), str("|".join(intersection)),
                                       str(len(unique_to_BUSTEDS_MH)), "|".join(unique_to_BUSTEDS_MH),
                                       str(len(unique_to_BUSTEDS)), "|".join(unique_to_BUSTEDS)])
    #end with
    
    
    # Update dict
    
    # In BUSTEDS-MH and BUSTEDS
    df_dict[basename].update({"++": len(intersection)})
    
    # In BUSTEDS-MH and NOT BUSTEDS
    #df_dict[basename].update({"+-": abs(len(ER_SITES_BUSTEDS_MH) - len(intersection))})
    df_dict[basename].update({"+-": len(unique_to_BUSTEDS_MH)})
    #df_dict[basename].update({"+-": len(ER_SITES_BUSTEDS_MH)})
    
    # NOT in BUSTEDS-MH but in BUSTEDS
    #df_dict[basename].update({"-+": abs(len(ER_SITES_BUSTEDS) - len(intersection))})
    df_dict[basename].update({"-+": len(unique_to_BUSTEDS)})
    #df_dict[basename].update({"-+": len(ER_SITES_BUSTEDS)})
    
    print()
    

# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/InfluenzaA.nex.BUSTEDS-MH.json
# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/InfluenzaA.nex.BUSTEDS.json
# Data loaded
# Calculating 'Unconstrained model' AICc delta, (BUSTEDS-MH) 23230.10669984552 and (BUSTEDS) 23242.49538542873
# ER Constrained: 329
# All BUSTEDS-MH ER Sites: [] 0
# All BUSTEDS ER Sites: ['1', '2', '3', '5', '11', '19', '21', '23', '31', '34', '36', '38', '41', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '57', '59', '62', '74', '75', '78', '80', '82', '86', '88', '91', '92', '94', '96', '102', '103', '106', '107', '112', '117', '121', '122', '124', '126', '128', '131', '133', '135', '137', '138', '140', '142', '144', '145', '146', '150', '156', '157', '158', '159', '162', '163', '164', '167', '168', '172', '173', '174', '175', '179', '182', '186', '188', '189', '190', '192', '193', '194', '196', '197', '198', '201', '202', '203', '212', '213', 